# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [2]:
# Setup
from __future__ import division, print_function, unicode_literals

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [3]:
import keras as kr

Using TensorFlow backend.


# Convolutional Neural Network

**Load Coarse-To-Fine Mappings**

In [149]:
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1
np.set_printoptions(threshold=np.inf)
#print(C2K_dot)

In [148]:
x = np.array([1,2,3,4,5])

y = np.array([[1, 1, 0, 0, 1],[0, 0, 1, 1, 0]])
print(np.dot(y,x))

[8 7]


**Import Dataset**

In [154]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

** Defining Structure**

In [138]:
def indep_layers(X):
    net = Conv2D(6, 2, strides=1, activation='relu', padding='same')(X)
    net = MaxPooling2D(pool_size=2, strides=2, padding='same')(net)
    net = Conv2D(3, 2, strides=1, activation='relu', padding='same')(net)
    net = Conv2D(3, 2, strides=1, activation='relu', padding='same')(net)
    net = Conv2D(3, 2, strides=1, activation='relu', padding='same')(net)
    net = Reshape([-1])(net)

    net = Dense(256, activation='relu')(net)
    net = Dropout(0.50)(net)
    net = Dense(128, activation='relu')(net)
    net = Dropout(0.25)(net)
    net = Dense(100, activation='softmax')(net)
    
    return net

In [156]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, multiply, dot
from keras.backend import constant
from keras.models import Model

coarse_categories = 20
fine_categories = 100

input_img = Input(shape=(32, 32, 3), dtype='float32', name='main_input')


## SHARED LAYERS
conv_shared = Conv2D(12, (4, 4), strides=(1, 1), padding='same', activation='relu')(input_img)
pool_shared = MaxPooling2D((2, 2), padding='same')(conv_shared)
shared_output = Dropout(0.25)(pool_shared)


## FOR COARSE CLASSIFIER
output_c = indep_layers(shared_output)
fine_to_coarse = constant(C2K_dot)
coarse_prob = dot([fine_to_coarse,output_c], 1)
probalistic_ave = dot([coarse_prob,fine_to_coarse], 0)

for i in range(20):
    
    ## TODO Thresholding
    
    fine_output = indep_layers(shared_output)
    
    ## TODO probabalistic averaging

model = Model(inputs=input_img, outputs=output_c)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [157]:
output_c = indep_layers(shared_output)
fine_to_coarse = constant(C2K_dot)
coarse_prob = dot([fine_to_coarse,output_c], 1)
probalistic_ave = dot([coarse_prob,fine_to_coarse], 0)
print(probalistic_ave)

Tensor("dot_64/ExpandDims:0", shape=(100, 1), dtype=float32)


**Fit Training Data**

In [158]:

## TODO EXTEND y_train to one-hot enconding
model.fit(x_train, y_train, epochs=1, batch_size=64)

ValueError: Error when checking model target: expected dense_222 to have shape (None, 100) but got array with shape (50000, 1)

**Fit on dataset delayed (for use in browsers)**

In [24]:
## TODO Update to new train variables

index = 45
model.save_weights('data/models/layer_'+str(index))
model.load_weights('data/models/layer_'+str(index), by_name=False)
model.fit([train_data1, weights[0], weights[1]], train_labels1_f, epochs=i+5, batch_size=64, initial_epoch=index)

Epoch 46/50
4500/4500 [==============================] - 4s - loss: 4.6052 - acc: 0.9796     

**Evaluate on testing set**

In [45]:
## TODO update to new test variables

model.evaluate([test_data_adj, test_weights[0], test_weights[1]], test_labels_adj, batch_size=64, verbose=1)

 960/1000 [===========================>..] - ETA: 0s

[23117.954390625, 0.998]